# Assignment 2: Card Draw

![Assignment 2 Image of Five Playing Cards](img/Card_Draw.png)

Author: Irene Kilgannon

Write a program that 'deals' i.e. prints out 5 cards from this API, https://deckofcardsapi.com/.

First shuffle the deck and get the deck_id. With the deck_id, get the cards. Print the value and suit of each card.

Extra marks: If the user has drawn a pair, triple, straight, or all of the same suit and congratulate the user.

In [294]:
# Import modules
import requests
import json
import pandas as pd
from emoji import emojize

## Shuffle The 'Cards'

In [295]:
shuffle_cards_url = "https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=1"

# Get data from the API
response = requests.get(shuffle_cards_url)

# Convert JSON response to Python dictionary
response_dict = response.json()

print('Deck ID:', response_dict['deck_id'])

Deck ID: px544sabdn9j


In [296]:
# Create a variable of the deck_id
deck_id =  response_dict['deck_id']

## Deal Five Cards

In [297]:
deal_cards_url = f"https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=5"

# Get data from the deal_cards_URL
response2 = requests.get(deal_cards_url)

# Convert JSON response to Python dictionary
deal_cards = response2.json()

# Print the value and suit of each card
for card in deal_cards['cards']:
    print(f"{card['value']} of {card['suit']}")

ACE of HEARTS
QUEEN of HEARTS
KING of DIAMONDS
ACE of CLUBS
6 of DIAMONDS


In [298]:
# Write data to a JSON file
with open ("data/deal_cards.json", "w") as fp:
    json.dump(deal_cards, fp)

## Extra Marks

If the user has drawn a pair, triple, straight, or all cards of the same suit and congratulate the user.

In [299]:
# Convert to a DataFrame
df = pd.DataFrame(deal_cards['cards'], columns= ['value', 'suit'])
df

,value,suit
0,ACE,HEARTS
1,QUEEN,HEARTS
2,KING,DIAMONDS
3,ACE,CLUBS
4,6,DIAMONDS


### Checking For a Pair Or Triple.

In [300]:
# Count the number of duplicate rows in the value column.
no_of_duplicated_values = df['value'].duplicated().sum()
print(f'No. of duplicated card values: {no_of_duplicated_values}')

No. of duplicated card values: 1


In [301]:
# Create a dataframe of the duplicated rows. 
df2 = df[df['value'].duplicated(keep = False)]

In [302]:
# Check for a pair.
if no_of_duplicated_values == 1:
    print(f"{emojize(":partying_face:")} Congratulations!!! You have a pair of {df2.iloc[0, 0]}'s.")

# Check for 2 pairs or a triple.
elif no_of_duplicated_values == 2:
    # Check for a triple
    if len(df2) == 2:
        print(f"{emojize(":party_popper:")} Congratulations!!! You have a triple of {df2.iloc[0, 0]}'s.")
    # Check for 2 pairs
    else:
        print(f"{emojize(":party_popper:")} WOW! Congratulations!! You have a two pairs {df2.iloc[0, 0]} and {df2.iloc[0, 2]}.")
else:
    print(f'{emojize(":disappointed_face:")} Unlucky, no pairs or triples. Better luck next time.')

🥳 Congratulations!!! You have a pair of ACE's.


### Checking for the Matching Suit.

In [303]:
# Check if all cards have the same suit.
if df['suit'].duplicated().sum() == 4:
    print(f'{emojize(":fireworks:")} All cards have the same suit. Congratulations! {emojize(":fireworks:")}')

### Checking for a Straight.

In [304]:
# For straight, need an ordered list of card values.
card_value_order = ['ACE', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'JACK', 'QUEEN', 'KING']

In [305]:
# Sort value column by the order in card_value_order.
# https://stackoverflow.com/questions/52784410/sort-column-in-pandas-dataframe-by-specific-order
df.value = pd.Categorical(df.value, categories= card_value_order)
df = df.sort_values(by ='value')
df

,value,suit
0,ACE,HEARTS
3,ACE,CLUBS
4,6,DIAMONDS
1,QUEEN,HEARTS
2,KING,DIAMONDS


In [306]:
# Create a list of the card values from the dataframe.
values = df.iloc[:5, 0].tolist()
values

['ACE', 'ACE', '6', 'QUEEN', 'KING']

In [307]:
# Check for the presence of the values list in card_values. 
# https://stackoverflow.com/questions/3313590/check-for-presence-of-a-sliced-list-in-python

n = len(values)

if any((values == card_value_order[i:i+n]) for i in range(n+1)):
    print("You have a straight.")
else:
    print(f"{emojize(":disappointed_face:")} No straight found.")

😞 No straight found.


## References

[Geeksforgeeks.org, response.json() - Python requests](https://www.geeksforgeeks.org/response-json-python-requests/)

[W3 schools.com, Python Requests Module](https://www.w3schools.com/python/module_requests.asp)

[Realpython.com, Python's Requests Library (Guide)](https://realpython.com/python-requests/)

[Stackoverflow.com, How to access a dictionary key value present inside a list](https://stackoverflow.com/questions/6521892/how-to-access-a-dictionary-key-value-present-inside-a-list)

[Geeksforgeeks.org, Accessing nested dictionaries in a list](https://www.geeksforgeeks.org/accessing-value-inside-python-nested-dictionaries/) 

[Datacamp.com, importing json data and working with APIs](https://campus.datacamp.com/courses/streamlined-data-ingestion-with-pandas/importing-json-data-and-working-with-apis?ex=4)

[Get a list from a dataframe](https://saturncloud.io/blog/how-to-get-a-list-from-a-pandas-dataframe-column/#:~:text=Best%20Practices-,Use%20the%20tolist()%20Method,of%20data%20into%20a%20list)